# Tutorial

This notebook will guide you through a simple estimation task with skillmodels. For this we use again the second example model from the CHS replication files, so we can re-use the model specification file written in the previous section. 

You will then learn how to use this model specification to simulate or estimate a model. Moreover, you will learn how you can use estimagic to impose additional constraints on your parameter vector. 

In [1]:
import mkl
mkl.set_num_threads(1)

4

Setting the mkl number of threads to 1 is actually faster than allowing for multithreading on small datasets. Note that this has to be done before numpy is imported. In our case numpy is never imported directly, but the import of pandas or SkillModel will import numpy.

In [2]:
import json
import pandas as pd
from skillmodels import SkillModel

First, load the model specification and prepare the dataset. In particular, we have to set the index of the dataset. Skillmodels will assume that the first index level identifies the individual and the second the period. The names of the levels are irrelevant. 

Also, it is good practice not to use floats in an index as this can lead to problems. 

In [3]:
with open("test_model2.json") as j:
    model_dict = json.load(j)
    
data = pd.read_stata("chs_test_ex2.dta")
for var in ["caseid", "period"]:
    data[var] = data[var].astype(int)
data.set_index(["caseid", "period"], inplace=True)
data.head()

index        y1        y2        y3        y4        y5  \
caseid period                                                            
1      0           0  1.909221  2.053261  1.679474  1.205891  2.195575   
       1           1  0.925990  1.828494  1.412966  1.235554  0.636243   
       2           2  1.957160  2.251265 -0.373637  2.875391  1.838843   
       3           3  1.236615  1.160494  3.001797  1.181049  2.170630   
       4           4  2.091614  0.664072  2.015088  1.193765  0.364270   

                     y6        y7        y8        y9       Q1  dy7  dy8  dy9  \
caseid period                                                                   
1      0       1.499965  0.873044  1.790903  1.191478  1.29912  1.0  1.0  1.0   
       1       1.534268  0.873044  1.790903  1.191478  1.29912  1.0  1.0  1.0   
       2       2.253621  0.873044  1.790903  1.191478  1.29912  1.0  1.0  1.0   
       3       1.125383  0.873044  1.790903  1.191478  1.29912  1.0  1.0  1.0   
       4       0.161003  0.873044  1.790903  1.191478  1.29912  1.0  1.0  1.0   

                     x1   x2   id  
caseid period                      
1      0       0.473032  1.0  0.0  
       1       0.473032  1.0  0.0  
       2       0.473032  1.0  0.0  
       3       0.473032  1.0  0.0  
       4       0.473032  1.0  0.0

Next we have to generate an instance of `SkillModel`

In [4]:
mod = SkillModel(model_dict=model_dict, dataset=data)

If you want to estimate a model, you can often greatly reduce the number of function evaluations needed during the optimization if you take some time to write down good start values. If you want to simulate a model, you will also need a parameter vector. 

Since skillmodels builds on estimagic, the parameter vector is not a numpy array but a pandas DataFrame. This DataFrame has a quite complicated MultiIndex that would be difficult to write down manually. Therefore, skillmodels has a helper function for that:

In [5]:
free, fixed = mod.start_params_helpers()
len(free)

208

You will typically only need the free parameters. Even the very simple example model has 208 of them! Next, let's save them as a csv file, so you can add your your start values in the value column and look briefly at the first few parameters.

In [6]:
free.to_csv("start_params_template.csv")
free.head()

value  lower  upper
category period name1 name2                        
delta    0      y1    constant    NaN   -inf    inf
                      x1          NaN   -inf    inf
                y2    constant    NaN   -inf    inf
                      x1          NaN   -inf    inf
                y3    constant    NaN   -inf    inf

The parameters have a MultiIndex with four levels. The first is the broad category of the parameters, i.e. distinguishs loadings, control parameters, transition parameters and so on. The second level is the period in which that parameter is used. The last two levels contain more information on the particular parameters. The four levels togethether should give you enough information to understand what each parameter means. 

Let's now assume you filled out the value column of the csv file with good start values and saved it in a file called ``start_params.csv``. Since this is an example model I can cheat and use very good start values, namely the ones from the CHS replication files. 

In [7]:
start_params = pd.read_csv("start_params.csv").set_index(
    ["category", "period", "name1", "name2"])
start_params.head()

value  lower  upper
category period name1 name2                           
delta    0      y1    constant  1.005455   -inf    inf
                      x1        1.001618   -inf    inf
                y2    constant  0.975992   -inf    inf
                      x1        1.031439   -inf    inf
                y3    constant  0.994139   -inf    inf

In [8]:
sim_observed, sim_latent = mod.simulate(params=start_params, nobs=1000)
sim_observed.head()

Q1  constant        x1        y1        y2        y3        y4  \
id period                                                                   
0  0      NaN       1.0  0.717256  2.567774  3.289202  2.669577  2.020750   
   1      NaN       1.0  0.717256  3.277900  1.775211  3.317693  2.071699   
   2      NaN       1.0  0.717256  0.821855  1.487925  2.931193  0.644986   
   3      NaN       1.0  0.717256  0.996576  1.430569  1.149163  2.072210   
   4      NaN       1.0  0.717256  0.268275  0.787743  0.765746  1.549459   

                 y5        y6        y7        y8        y9  
id period                                                    
0  0       1.566045  2.607329  2.633494  3.626662  1.316714  
   1       1.848631  2.080483       NaN       NaN       NaN  
   2       0.808183  1.362376       NaN       NaN       NaN  
   3       1.770856  1.809766       NaN       NaN       NaN  
   4       1.243876  2.049762       NaN       NaN       NaN

Of course, you would usually rather simulate a dataset at parameters you estimated and not at ones you invented. For this you have to estimate the model first. This is as easy as typing:

``mod.fit(start_params=start_params)``

but to get the same values as CHS we will have to do a little more work. The reason is that on top of the many constraints skillmodels generates atuomatically from the model specification, CHS impose two more constraints:

1. The constant in the linear transition equation is fixed to 0
2. The initial mean of the states is not estimated but assumed to be zero. 

Fortunately, estimagic makes it easy to express such constraints:

In [9]:
additional_constraints = [
    {"loc": ("trans", 0, "fac2", "constant"), "type": "fixed", "value": 0},
    {"loc": "x", "type": "fixed", "value": 0}
]

If you are not familiar with estimagic constraints, Check out the [documentation](https://estimagic.readthedocs.io/en/master/optimization/constraints.html). Next we can call the fit method of SkillModel. This is commented out because it takes quite long. 

In [10]:
# info, params = mod.fit(
#     start_params=start_params, 
#     user_constraints=additional_constraints,
#     dashboard=True,
#     db_options={"rollover": 20000, "evaluations_to_skip": 208},
#     algo_options={"ftol": 1e-8}
# )

The arguments of fit are all passed through to estimagic's maximize function. You can find more information on them in the [estimagic documentation](https://estimagic.readthedocs.io/en/master/optimization/index.html)